<a href="https://colab.research.google.com/github/pachterlab/seqspec/blob/libspec/examples/seqspec-dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf seqspec
!git clone https://github.com/pachterlab/seqspec
!cd seqspec && git checkout -b libspec origin/libspec && pip install --quiet .

Cloning into 'seqspec'...
remote: Enumerating objects: 1609, done.
remote: Counting objects: 100% (786/786), done.
remote: Compressing objects: 100% (342/342), done.
remote: Total 1609 (delta 495), reused 699 (delta 438), pack-reused 823
Receiving objects: 100% (1609/1609), 6.42 MiB | 9.70 MiB/s, done.
Resolving deltas: 100% (1016/1016), done.
Branch 'libspec' set up to track remote branch 'libspec' from 'origin'.
Switched to a new branch 'libspec'
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.8 MB/s eta 0:00:00


In [2]:
!seqspec

usage: seqspec [-h] <CMD> ...

seqspec 0.1.1: Format sequence specification files

positional arguments:
  <CMD>
    check     validate seqspec file
    find      find regions in a seqspec file
    format    format seqspec file
    genbank   get genbank about seqspec file
    index     index regions in a seqspec file
    info      get info about seqspec file
    init      init a seqspec file
    modify    modify region attributes
    onlist    get onlist file for specific regions
    print     print seqspec file
    split     split seqspec into modalities
    version   Get seqspec version and seqspec file version

options:
  -h, --help  show this help message and exit


In [3]:
!head seqspec/docs/assays/illumina_truseq_dual.spec.yaml

!Assay
seqspec_version: 0.0.0
assay_id: Illumina-novaseq-truseq-dual-index
name: Example assay
doi: https://support.illumina.com/content/dam/illumina-support/documents/documentation/system_documentation/miseq/indexed-sequencing-overview-guide-15057455-08.pdf
date: 21 February 2024
description: Example seqspec for an assay
modalities:
- rna
lib_struct: null


In [4]:
!seqspec print seqspec/docs/assays/illumina_truseq_dual.spec.yaml

                                        ┌─'illumina_p5:29'
                                        ├─'truseq_read1:10'
                                        ├─'insert:150'
─────────────────── ──rna───────────────┤
                                        ├─'truseq_read2:34'
                                        ├─'index7:8'
                                        └─'illumina_p7:24'


In [5]:
!seqspec check seqspec/docs/assays/illumina_truseq_dual.spec.yaml

[error 1] None is not of type 'string' in spec['lib_struct']
[error 2] 'insert' is not one of ['atac', 'barcode', 'cdna', 'crispr', 'custom_primer', 'dna', 'fastq', 'fastq_link', 'gdna', 'hic', 'illumina_p5', 'illumina_p7', 'index5', 'index7', 'linker', 'ME1', 'ME2', 'methyl', 'named', 'nextera_read1', 'nextera_read2', 'poly_A', 'poly_G', 'poly_T', 'poly_C', 'protein', 'rna', 's5', 's7', 'tag', 'truseq_read1', 'truseq_read2', 'umi'] in spec['library_spec'][0]['regions'][2]['region_type']
[error 3] None is not of type 'string' in spec['library_spec'][0]['regions'][4]['onlist']['md5']
[error 3] index7_onlist.txt does not exist
[error 4] R1.fastq.gz file does not exist
[error 5] I1.fastq.gz file does not exist
[error 6] I2.fastq.gz file does not exist
[error 7] R2.fastq.gz file does not exist
[error 8] 'truseq_read1' sequence 'TCTTTCCCTACACGACGCTCTTCCGATCT' has length 29, expected range (10, 10)


In [6]:
!seqspec index -m rna -r R1.fastq.gz seqspec/docs/assays/illumina_truseq_dual.spec.yaml

R1.fastq.gz	insert	0	98


In [7]:
!seqspec index --rev -m rna -r R2.fastq.gz seqspec/docs/assays/illumina_truseq_dual.spec.yaml

R2.fastq.gz	insert	0	98


In [8]:
!seqspec find -m rna -r insert seqspec/docs/assays/illumina_truseq_dual.spec.yaml

- !Region
  region_id: insert
  region_type: insert
  name: Custom insert
  sequence_type: random
  sequence: XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
  min_len: 1
  max_len: 150
  onlist: null
  regions: null
  parent_id: rna



# Code testing

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.dates as mdates

from seqspec.utils import load_spec, get_cuts
from seqspec.seqspec_index import run_index
from seqspec.seqspec_find import run_find
import os

fsize=15

plt.rcParams.update({'font.size': fsize})
%config InlineBackend.figure_format = 'retina'

In [10]:
spec = load_spec("seqspec/docs/assays/element_adept_truseq_dual.spec.yaml")

# modality
mode = "rna"

# library and sequence spec
libspec = spec.get_libspec(mode)
seqspec = spec.get_seqspec(mode)


In [25]:
from seqspec.utils import get_cuts

def complement_nucleotide(nucleotide):
    complements = {
        'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
        'R': 'Y', 'Y': 'R', 'S': 'S', 'W': 'W',
        'K': 'M', 'M': 'K', 'B': 'V', 'D': 'H',
        'V': 'B', 'H': 'D', 'N': 'N', 'X':'X'
    }
    return complements.get(nucleotide, 'N')  # Default to 'N' if nucleotide is not recognized

def complement_sequence(sequence):
    return ''.join(complement_nucleotide(n) for n in sequence.upper())


def run_print_both(spec, modality):
  libspec = spec.get_libspec(mode)
  seqspec = spec.get_seqspec(mode)

  p = []
  n = []
  leaves = libspec.get_leaves()
  cuts = get_cuts(leaves)
  for idx, read in enumerate(seqspec, 1):
    read_len = read.max_len
    read_id = read.read_id
    primer_id = read.primer_id
    primer_idx = [i for i, l in enumerate(leaves) if l.region_id == primer_id][0]
    primer_pos = cuts[primer_idx]
    if read.strand == "pos":

      wsl = primer_pos[1]-1
      ws = wsl*' '

      arrowl = read_len-1
      arrow = arrowl*'-'

      p.append(f"{ws}|{arrow}>({idx}) {read_id}")
    elif read.strand == "neg":
      wsl = primer_pos[0] - read_len
      ws = wsl*' '

      arrowl = read_len-1
      arrow = arrowl*'-'

      n.append(f"{ws}<{arrow}|({idx}) {read_id}")

      s = "\n".join([
          "\n".join(p),
          libspec.sequence,
          complement_sequence(libspec.sequence),
          "\n".join(n)
          ])
      return s

In [26]:
s = run_print_both(spec, "rna")

In [27]:
print(s)

                                                                                                                                                      |-------->(1) I1.fastq.gz
                            |-------->(2) I2.fastq.gz
                                                                  |------------------------------------------------->(3) R1.fastq.gz
AATGATACGGCGACCACCGAGATCTACACNNNNNNNNNTCTTTCCCTACACGACGCTCTTCCGATCTXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXAGATCGGAAGAGCACACGTCTGAACTCCAGTCACNNNNNNNNNATCTCGTATGCCGTCTTCTGCTTG
TTACTATGCCGCTGGTGGCTCTAGATGTGNNNNNNNNNAGAAAGGGATGTGCTGCGAGAAGGCTAGAXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXTCTAGCCTTCTCGTGTGCAGACTTGAGGTCAGTGNNNNNNNNNTAGAGCATACGGCAGAAGACGAAC
                                                                   <-------------------------------------------------|(4) R2.fastq.gz
